In [112]:
import numpy as np
import pandas as pd
import tensorflow
import torch
from nltk.tokenize import word_tokenize
from nltk import download


#For BARD
from transformers import pipeline

#For mT5-multilingual-XLSum
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


<h5>Articles

In [113]:
ARTICLE1 = '''There are different contemporary issues in the United States healthcare system that professionals have to face. As discussed by C-suite and director-level executives that are the members of HealthCare Executive Group (HCEG) organization, some of the problems are costs and transparency, consumer experience, data and analytics, and holistic individual health (MHE Staff, 2019). The crucial issue for the US healthcare system nowadays is data and analytics, including labs, sensors, mHealth, Internet of Things (IoT) technologies, that members of the nursing discipline can help to address.
The emergence of nursing informatics education and specialization offer many opportunities to facilitate the process of IT transformation at healthcare organizations. It is reported that nurses have an essential impact on the data and analytics field because they usually use Electronic Health Records (EHR) and have an in-depth knowledge of healthcare workflows (Morgan, 2017). Analysts state that there is a high demand for nursing professionals in the US who can analyze a vast amount of data and contribute to enhancing healthcare quality, patient protection, and efficiency. Moreover, specialized nurses can contribute their knowledge of patient behaviors to answer clinical questions, using big data analytics gathered with the usage of wearable devices and sensors, EHR, internet, and genome information (Gleason & Himmelfarb, 2017).
To make a conclusion, one can state that today is the best time to turn challenges into opportunities, noting that professionals might significantly contribute to the development of IT systems in clinical organizations and enhance the quality of patient treatment. Nursing executives in the US healthcare system might substantially improve the utilization of technologies and provide education for nurses to effectively process the data gathered from different resources and extract valuable insights about patients health. These activities can be achieved by continuous education and usage of emerging technologies to analyze data by nurses.'''


ARTICLE2 = '''The major change that occurred in my clinical settings is the adoption of Electronic Health Records (EHR). The complexity of the change is explained by the fact that its success is largely predetermined by the willingness of the staff to accept the new technology. Otherwise its meaningful use cannot be ensured (Murdoch & Detsky, 2013). I believe that the change was successful due to the efforts made by the nurse facilitator, who was responsible for the transfer.
The effectiveness of the change was predetermined by her ability to prepare nurses for the upcoming change process and overcome their resistance. Furthermore, the success was also due to the ability of the hospital personnel to cooperate, share experience and information, applying knowledge and skills, and support one another in their striving to achieve the common goal. The adoption of the technology was facilitated by this human factor as every member of the team acted as a change agent, trying to improve the process of organizing, storing, and sharing patient data with all the parties involved in treatment.
Last but not least, the proper selection of a change model contributed a lot to the outcome of the transition to a new technology. It has been proven by research that the selection of an efficient framework that would make it possible to transfer theoretical knowledge to clinical practice is one of the key preliminary steps in the change process. Lewins Change Theory was selected as a change model by the hospital administration due to the fact that it provides a clear method to follow the initial design.
The main underlying concepts are field and force: The former presupposes that in case one of the elements is altered, the whole system is affected, while the other provided necessary motivation or shift towards transformation (Shirey, 2013).'''


ARTICLE3 = '''There are many principles according to which nurses and patients can develop their relations. Each principle is a unique combination of ethical and legal aspects. They help to define the way for nurses to react to different patients behaviors and choose the most appropriate solutions to the possible problems and challenges. The current article discusses the peculiarities of the informed consent process within the frames of which a patient and its surgeon discuss the details of a coming operation. However, the author of the article underlines the fact that nurses have to be also defined as considerable participants of such kind of agreement. Nurses have to perform a function of a patient advocate, who has “an entrusted interest in fully understanding the legal and ethical considerations of the informed consent process”.
The article has several logical sections in which Menendez discloses the peculiarities of the relations between nurses and patients and defines the functions nurses have to perform in regards to their patients. With the help of a literature review, it becomes apparent that the necessity to identify the risks, benefits, and possible alternatives of surgeries has a long history. Still, the role of nurses in this process has been recently identified. The point is that nurses have all legal rights to facilitate patient autonomy and support the decision of a patient taking into consideration the current physical conditions.
In other words, this article helps to understand the importance of the positions any nurse can have. At the same time, the author proves that nurses have to develop appropriate relations with surgeons to make sure that the priority of patient autonomy does not influence the patients health condition. The informed consent process helps to predetermine the development of the relations between a patient, surgeon, and nurse.'''


ARTICLE4 = '''When studying the history of China, many put a focus on innovations that gave a start to the development of Western civilizations. Despite the fact that China was very advanced in agricultural and technological productivity in the time period leading up to the fourteenth century, the country stopped its development and fell behind in comparison with the West. It is important to mention that the sphere of innovation in the postmodern period was characterized by remarkable inventions such as paper, gunpowder, and the magnetic compass; items that became crucial components for facilitating progress in Europe. Many explained Chinas developmental stagnation by the concept of “high-level equilibrium trap”, which was characterized by the lack of human capital and entrepreneurship, restricted markets, and political hazards 
The explanation as to why the Industrial Revolution did not originate in China is associated with the nature of invention and discovery. For example, in the premodern time period, innovations were developed from experience acquired from many years of performing the same tasks. Contrary to this, the postmodern time period was characterized by innovation stemming from experiments. Therefore, China fell behind in its innovational development because it did not make a change towards experiment-based innovation and continued grounding the technological advancements on experience.
Needhams paradox of why Chinas technology was so far ahead from Western civilizations but failed in modern times is explained by many theories. Despite the fact that the large population of China facilitated innovations and development in premodern times, the country collapsed on its way towards advancement because of the failure to integrate the scientific component into the creation of new inventions (Lin 286). This means that the country had to be better equipped with important technology and adaptive innovations that would facilitate domestic research and put the country ahead of others.'''


ARTICLE5 = '''A fast food restaurant is a service firm that produces food products and serves them to its customers. In this scenario, the fast food restaurant which is experiencing a booming business, is looking for ways to increase its capacity to serve more customers. The management and the day supervisors have then come up with two options, which are either add more workers or add a new grilling machine. Let us look at the viability of the two options.
The first option which is to add a new grill and French fry machine, which are capital goods, will increase the restaurants ability to produce more food products which are produced with the aid of the machines. By adding its capital goods, the management will be able to serve more customers with the products, as they have the required equipment which means they will be able to meet their demands. By adding more equipment, the restaurant may face the problem of having a shortage of workers needed to operate these machines.
The second option which is to add more workers means that the company which is a service firm is able to serve its customers when demand is high but will be facing the problem of shortage of the products being sold. If more labor than capital is hired, the restaurant will be having a problem of excess labor hours which will be costly to the restaurants and will affect its profitability.
The best alternative among the two is to add a new grill and a French fry machine, which will enable the restaurant to produce more food products, which are the product its customers want. With the ability to meet the demand of the food product, the firm will have more sales and will be able to meet its profitability goals.'''


<h5>Models

In [114]:
def models(Article,summ_txt):
    #Model 1
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    s = summarizer(Article, max_length=130, min_length=30, do_sample=False)
    summ_txt.append(s[0]['summary_text'])
    
    #Model 2
    summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum")
    s = summarizer(Article)
    summ_txt.append(s[0]['summary_text'])
    
    #Model 3
    summarizer = pipeline("summarization", model="Falconsai/text_summarization")
    s = summarizer(Article)
    summ_txt.append(s[0]['summary_text'])
    
    #Model 4
    summarizer = pipeline("summarization", model="knkarthick/MEETING_SUMMARY")
    s = summarizer(Article)
    summ_txt.append(s[0]['summary_text'])
    
    #Model 5
    summarizer = pipeline("summarization", model="lidiya/bart-large-xsum-samsum")
    s = summarizer(Article)
    summ_txt.append(s[0]['summary_text'])
    
    return (summ_txt)


<h5>Test Comparison Score

In [115]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu
from fuzzywuzzy import fuzz

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalpha()]
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

def preprocess_text_for_J(text):
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalpha()]
    words = [word for word in words if word not in stop_words]
    return set(words)

def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

def rouge_score(reference, candidate):
    rouge = Rouge()
    scores = rouge.get_scores(candidate, reference)
    return scores[0]['rouge-l']['f']

def cosine_similarity_score(text1, text2):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([text1, text2])
    return cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]

def bleu_score(reference, candidate):
    reference_tokens = word_tokenize(reference)
    candidate_tokens = word_tokenize(candidate)
    return sentence_bleu([reference_tokens], candidate_tokens)

def monge_elkan_similarity(string1, string2):
    return fuzz.token_set_ratio(string1, string2) / 100.0

# Example usage:


def score_cal(long_text,summaries):

    # Preprocess long text
    preprocessed_long_text = preprocess_text(long_text)
    preprocessed_long_text_for_J = preprocess_text_for_J(long_text)

    # Calculate similarity scores
    jaccard_similarities = [jaccard_similarity(preprocess_text_for_J(summary), preprocessed_long_text_for_J) for summary in summaries]
    rouge_scores = [rouge_score(preprocessed_long_text, preprocess_text(summary)) for summary in summaries]
    cosine_similarities = [cosine_similarity_score(preprocessed_long_text, preprocess_text(summary)) for summary in summaries]
    bleu_scores = [bleu_score(preprocessed_long_text, preprocess_text(summary)) for summary in summaries]
    monge_elkan_scores = [fuzz.token_set_ratio(preprocess_text(summary), preprocessed_long_text) / 100.0 for summary in summaries]

    scor_mat = [np.round(i,5) for i in (jaccard_similarities,rouge_scores,cosine_similarities,bleu_scores,monge_elkan_scores)]

    return(scor_mat)


[nltk_data] Downloading package punkt to C:\Users\SUNIL
[nltk_data]     KUMAR\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\SUNIL
[nltk_data]     KUMAR\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<h5>Topsis

In [116]:
def topsis(Data,Weights,Impacts):
    df = Data
    Items_extracted = Data[Data.columns[0]]
    Data = Data.drop(Data.columns[0],axis=1)
    
    # Taking Root of Sum of Squres of each column and dividing it with individuals
    for index,col in enumerate(Data.columns):
        Root_Of_Sq_Sum = np.sqrt(sum(np.square(Data[col])))  #taking Root of Sum of Squares
        Data[col] = (Data[col]*Weights[index])/Root_Of_Sq_Sum  #Dividing with RSS and Multiplying with weight individuals
        
    #Calculating V_plus and V_neg of each column    
    V_pos = [np.max(Data[col]) if Impacts[index] == '+' else round(np.min(Data[col]), 4) for index, col in enumerate(Data.columns)]
    V_neg = [np.min(Data[col]) if Impacts[index] == '+' else round(np.max(Data[col]), 4) for index, col in enumerate(Data.columns)]
     
    #Calculating Euclidean distance 
    S_pos = [np.sqrt(sum(np.square(x - y) for x, y in zip(V_pos, Data.iloc[i]))) for i in range(Data.shape[0])]
    S_neg = [np.sqrt(sum(np.square(x - y) for x, y in zip(V_neg, Data.iloc[i]))) for i in range(Data.shape[0])]
    
    #Calculating Performance
    P = [round(y/(x+y),5) for x,y in zip(S_pos,S_neg)]
          
    #Creating Dataframe only with Performance and Ranks      
    performance_Data = pd.DataFrame(data ={'Items':Items_extracted , 'Performance':P})
    performance_Data['Rank'] = performance_Data['Performance'].rank(ascending=False)
    
    #Dataframe fith Performance and older data
    df['Performance'] = P
    df['Rank'] = df['Performance'].rank(ascending=False)

    return (df)
        

In [117]:
articles = [ARTICLE1,ARTICLE2,ARTICLE3,ARTICLE4,ARTICLE5]
weights = [1,1,1,1,1]
Impacts = ['+','+','+','+','+']
data_tables = []
for i in range(len(articles)):
    summary = []
    models(articles[i],summary)
    scores = score_cal(articles[i],summary)
    
    dict = {}
    dict['Models'] = ['M1','M2','M3','M4','M5']
    dict['jaccard_similarity'] = scores[0]
    dict['rouge_score'] = scores[1]
    dict['cosine_similarities'] = scores[2]
    dict['bleu_scores'] = scores[3]
    dict['monge_elkan_scores'] = scores[4]
      
    df = pd.DataFrame(dict)    
    print(f'TEXT{i+1} - \n')
    df = topsis(df,weights,Impacts)
    data_tables.append(df)
    print(df)
    print('\n\n')
    

TEXT1 - 

  Models  jaccard_similarity  rouge_score  cosine_similarities  bleu_scores  \
0     M1             0.16667      0.28571              0.41993      0.00070   
1     M2             0.22222      0.36364              0.46860      0.00836   
2     M3             0.23622      0.38217              0.55076      0.01257   
3     M4             0.12598      0.19580              0.34696      0.00003   
4     M5             0.11111      0.18571              0.37132      0.00000   

   monge_elkan_scores  Performance  Rank  
0                1.00      0.21262   3.0  
1                1.00      0.69988   2.0  
2                0.99      0.99534   1.0  
3                0.98      0.04086   4.0  
4                1.00      0.02697   5.0  



TEXT2 - 

  Models  jaccard_similarity  rouge_score  cosine_similarities  bleu_scores  \
0     M1             0.19084      0.32051              0.51509      0.00733   
1     M2             0.35115      0.51977              0.61297      0.09957   
2     M

In [120]:
data_tables[0]

,Models,jaccard_similarity,rouge_score,cosine_similarities,bleu_scores,monge_elkan_scores,Performance,Rank
0,M1,0.16667,0.28571,0.41993,0.00070,1.00,0.21262,3.0
1,M2,0.22222,0.36364,0.46860,0.00836,1.00,0.69988,2.0
2,M3,0.23622,0.38217,0.55076,0.01257,0.99,0.99534,1.0
3,M4,0.12598,0.19580,0.34696,0.00003,0.98,0.04086,4.0
4,M5,0.11111,0.18571,0.37132,0.00000,1.00,0.02697,5.0


In [121]:
data_tables[1]

,Models,jaccard_similarity,rouge_score,cosine_similarities,bleu_scores,monge_elkan_scores,Performance,Rank
0,M1,0.19084,0.32051,0.51509,0.00733,1.00,0.13969,2.0
1,M2,0.35115,0.51977,0.61297,0.09957,1.00,1.00000,1.0
2,M3,0.20611,0.34177,0.46980,0.00897,1.00,0.13242,3.0
3,M4,0.18045,0.30573,0.48811,0.00736,0.96,0.11607,4.0
4,M5,0.17424,0.29677,0.34797,0.00354,0.98,0.00833,5.0


In [122]:
data_tables[2]

,Models,jaccard_similarity,rouge_score,cosine_similarities,bleu_scores,monge_elkan_scores,Performance,Rank
0,M1,0.22018,0.25564,0.60482,0.00535,1.00,0.34907,3.0
1,M2,0.23853,0.38519,0.51571,0.00850,1.00,0.62003,2.0
2,M3,0.17431,0.29687,0.45115,0.00178,1.00,0.07528,5.0
3,M4,0.19266,0.32308,0.50226,0.00270,1.00,0.16165,4.0
4,M5,0.24545,0.32117,0.62324,0.01301,0.99,0.88727,1.0


In [123]:
data_tables[3]

,Models,jaccard_similarity,rouge_score,cosine_similarities,bleu_scores,monge_elkan_scores,Performance,Rank
0,M1,0.19835,0.33103,0.47358,0.00345,1.0,0.25684,2.0
1,M2,0.28099,0.43871,0.56900,0.02906,1.0,1.00000,1.0
2,M3,0.19008,0.31944,0.37485,0.00193,1.0,0.21000,3.0
3,M4,0.14050,0.15942,0.42502,0.00008,1.0,0.04241,5.0
4,M5,0.14876,0.25899,0.39233,0.00026,1.0,0.11822,4.0


In [124]:
data_tables[4]

,Models,jaccard_similarity,rouge_score,cosine_similarities,bleu_scores,monge_elkan_scores,Performance,Rank
0,M1,0.32099,0.48598,0.56960,0.01791,1.00,0.80742,2.0
1,M2,0.34568,0.49541,0.62436,0.02208,1.00,1.00000,1.0
2,M3,0.29630,0.45714,0.42951,0.00701,1.00,0.35046,3.0
3,M4,0.20238,0.31683,0.39304,0.00097,0.95,0.00000,5.0
4,M5,0.21687,0.33663,0.45614,0.00114,0.97,0.07736,4.0


In [144]:
for i in range(5):
    print(f'Text{i+1} - ')
    df = data_tables[i]
    bst = df[df['Rank'] == 1]['Models']
    print('Best Model is ',list(bst)[0])
    print('\n')

Text1 - 
Best Model is  M3


Text2 - 
Best Model is  M2


Text3 - 
Best Model is  M5


Text4 - 
Best Model is  M2


Text5 - 
Best Model is  M2


